### Import Libraries and Initialisation

In [1]:
import requests
from base64 import b64encode
import json
import pandas as pd
from datetime import datetime, date, timedelta

import sys
import yaml as yml
import pyodbc as cnn

import re
from dateutil.relativedelta import relativedelta
from tqdm import tqdm, trange
   
apikey = "fz/ocDZn9rHm9ZAupGsDNjoBZGSnHO+8vmU/xWiPPvvFzPE0STJnTbnvu6/FvyOpUhp/TBu0txZ4xZYz6J8iyxkUDkBowVTiDflAvH0qdi5NzWSW+iq8L7j77/Qg77/6Wkj1JvxGkBTOtFOpV9saGw=="
clientId = "33604187fc197c87eb8d2f553935ef8e"

page = 1
pagesize = 1000

# Encode the credentials with base64
username = apikey
password = ''
encoded_credentials = b64encode(bytes(f'{username}:{password}',encoding='ascii')).decode('ascii')

# Use Basic Authorization
auth_header = f'Basic {encoded_credentials}'
print(f'Auth header: {auth_header}')

# Open the config file
with open(r'../dbconfig.yaml') as file:
    config = yml.full_load(file)

Auth header: Basic Znovb2NEWm45ckhtOVpBdXBHc0ROam9CWkdTbkhPKzh2bVUveFdpUFB2dkZ6UEUwU1RKblRibnZ1Ni9GdnlPcFVocC9UQnUwdHhaNHhaWXo2SjhpeXhrVURrQm93VlRpRGZsQXZIMHFkaTVOeldTVytpcThMN2o3Ny9RZzc3LzZXa2oxSnZ4R2tCVE90Rk9wVjlzYUd3PT06


### Common Functions


### Establish Database Connection

In [3]:
# Establish the connection with database server
conn = cnn.connect('DRIVER='+config['instances']['driver']+
                   ';SERVER='+config['instances']['servername']+
                   ';DATABASE='+config['instances']['database']+
                   ';UID='+config['instances']['username']+
                   ';PWD='+config['instances']['password'])

conn.autocommit = False

#check loading configuration
#METHOD: full/incremental
cfg_load_method = config['loading']['method']

###  Initial Objects Declaration

In [5]:
campaign_email_action_list = ["opens", "clicks", "unsubscribes", "bounces", "spam"]

n_past_days = config['campaign']['campaign_active_period_length']
params = {"page": page, "pagesize": pagesize, "orderfield": "email", "orderdirection": "asc" }
desired_columns = ["campaignId", "EmailAddress", "Date"]

final_campaign_action_data = {"opens": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_opens"},
                              "clicks":{"data": pd.DataFrame(), "destination_table_name": "campaign_email_clicks"},
                              "unsubscribes": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_unsubscribes"},
                              "bounces": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_bounces"},
                              "spam": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_spam"} }

#initiate
start_proc_time = datetime.now()

#START_DATE_FULL_LOAD: starting date of data to be loaded (if method is full-load)
cfg_start_date = config['loading']['start_date_full_load']

#DAY_UPDATE_LAG: number of days of data lag
cfg_update_lag = config['loading']['day_update_lag']


#calculate end date
cfg_end_date = datetime.now() - relativedelta(days = cfg_update_lag)

# 
end_proc_time = datetime.now()

cfg_loading_method = config['loading']['method']

KeyError: 'campaign'

In [4]:
### Test
campaign_detail_url = f'https://api.createsend.com/api/v3.2/admins.json'
response = requests.get(campaign_detail_url, headers={'Authorization': auth_header } ) 
response_campaign_list = response.json()
df_campaign_list = pd.DataFrame.from_dict(response_campaign_list)
df_campaign_list

,EmailAddress,Name,Status
0,Hnin.Tin@nswcc.org.au,Hnin,Active
1,deepuinuk@gmail.com,Sree,Active


In [56]:
### Get the sent campaign
def getAllSentCampaigns(clientId):
    campaign_detail_url = f'https://api.createsend.com/api/v3.2/clients/{clientId}/campaigns.json'
    response = requests.get(campaign_detail_url, headers={'Authorization': auth_header } ) 
    response_campaign_list = response.json()
    df_campaign_list = pd.DataFrame.from_dict(response_campaign_list)
    return df_campaign_list


def get_datetime_Ndays_ago(n):
    N_days_ago_datetime = datetime.now() - timedelta(days=n)
    return N_days_ago_datetime.strftime("%Y-%m-%d %H:%M")


def getFilteredActiveCampaign(df_campaign_list): 
    '''
    The function filters the campaigns which are still within 120 days since the email sent date.
    Basically, get all the still-active campaigns
    '''
    active_campaign_day = config['campaign']['campaign_active_period_length']
    active_period = get_datetime_Ndays_ago(active_period_length)

    all_active_campaign = df_campaign_list['SentDate'] > active_period 
    campaigns_on_active_period = df_campaign_list.loc[all_active_campaign]
    return campaigns_on_active_period


def insert_into_db(row, cursor, table):
    cursor.execute(f'  INSERT INTO {table}'
                   '  (campaign_id, email_address, action_date ) VALUES   (?,?,?) ',
                   row['campaignId'], row['EmailAddress'], row['Date'])
    
def get_next_NDays(dateFrom, n):
    datetime_object = datetime.strptime(dateFrom, '%Y-%m-%d %H:%M:%S')
    next_N_days_datetime = datetime_object + timedelta(days=n)
    return next_N_days_datetime.strftime("%Y-%m-%d %H:%M")


'2021-06-16 13:15'

### Data Acquistion through Campaign Monitor API

In [5]:
final_campaign_action_data = {"opens": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_opens"},
                              "clicks":{"data": pd.DataFrame(), "destination_table_name": "campaign_email_clicks"},
                              "unsubscribes": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_unsubscribes"},
                              "bounces": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_bounces"},
                              "spam": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_spam"} }


df_all_sent_campaigns_list = getAllSentCampaigns(clientId)
df_all_sent_campaigns_list
# If full-load, download all the historical past campaigns data
n_past_days = 50
if (cfg_loading_method == 'full'):
    campaigns_on_active_period = df_all_sent_campaigns_list
elif (cfg_loading_method == 'incremental'):
    campaigns_on_active_period = getFilteredActiveCampaign(df_all_sent_campaigns_list)

campaigns_on_active_period

for campaign_id in campaigns_on_active_period['CampaignID']:
    
    campaign = campaigns_on_active_period[campaigns_on_active_period['CampaignID'] == campaign_id]
       
    for action in campaign_email_action_list:

        url_campaign_opens = f"https://api.createsend.com/api/v3.2/campaigns/{campaign_id}/{action}.json"
        df_campaign_email_action = pd.DataFrame()
        all_records = []
        
        for x in range(n_past_days):

             N_days_ago_datetime = get_datetime_Ndays_ago(x)
             # Pass N days if not full load
             params["date"] = N_days_ago_datetime
             response = requests.get(url_campaign_opens, headers={'Authorization': auth_header }, params = params) 
             #print(response.json())
             response_campaign_action = response.json()
             if(len(response_campaign_action["Results"]) == 0): continue
             #print("Data exists")
             response_dataframe = pd.DataFrame.from_dict(response_campaign_action["Results"])
             response_dataframe["campaignId"] = campaign_id
             response_dataframe = response_dataframe[desired_columns]
             df_campaign_email_action = df_campaign_email_action.append(response_dataframe, ignore_index=True, sort=False) 
        
        if(len(df_campaign_email_action.index) > 0):           
            campaign_email_action_dataset = df_campaign_email_action.drop_duplicates()
            final_campaign_action_data[action]["data"] = final_campaign_action_data[action]["data"].append(campaign_email_action_dataset, ignore_index=True, sort=False) 

NameError: name 'getAllSentCampaigns' is not defined

In [35]:
final_campaign_action_data

{'opens': {'data': Empty DataFrame
  Columns: []
  Index: [], 'destination_table_name': 'campaign_email_opens'},
 'clicks': {'data': Empty DataFrame
  Columns: []
  Index: [], 'destination_table_name': 'campaign_email_clicks'},
 'unsubscribes': {'data': Empty DataFrame
  Columns: []
  Index: [], 'destination_table_name': 'campaign_email_unsubscribes'},
 'bounces': {'data': Empty DataFrame
  Columns: []
  Index: [], 'destination_table_name': 'campaign_email_bounces'},
 'spam': {'data': Empty DataFrame
  Columns: []
  Index: [], 'destination_table_name': 'campaign_email_spam'}}

In [8]:
print("Total Number of Records", campaingn_opens_dataset.shape)

Total Number of Records (10, 3)


In [34]:
final_campaign_action_data

{'opens': {'data': Empty DataFrame
  Columns: []
  Index: [], 'destination_table_name': 'campaign_email_opens'},
 'clicks': {'data': Empty DataFrame
  Columns: []
  Index: [], 'destination_table_name': 'campaign_email_clicks'},
 'unsubscribes': {'data': Empty DataFrame
  Columns: []
  Index: [], 'destination_table_name': 'campaign_email_unsubscribes'},
 'bounces': {'data': Empty DataFrame
  Columns: []
  Index: [], 'destination_table_name': 'campaign_email_bounces'},
 'spam': {'data': Empty DataFrame
  Columns: []
  Index: [], 'destination_table_name': 'campaign_email_spam'}}

### Insert Into Database


In [33]:
for action in campaign_email_action_list:
        
        cursor = conn.cursor()
        table_name = final_campaign_action_data[action]["destination_table_name"] 
        #dbfun_name = final_campaign_action_data[action]["db_function"] 
        campaign_data = final_campaign_action_data[action]["data"]
        if (campaign_data.empty) : continue
        try:
            #if full-load, delete all the records from the table
            if (cfg_load_method == 'full'):
                cursor.execute(f'DELETE FROM {table_name}')
            print("action", action)
            #insert dataframe to DB
            campaign_data.apply(insert_into_db, cursor = cursor, axis = 1, table = table_name)
            
            cursor.execute('INSERT INTO '
                   '       log_success_load '
                   '       (load_date, method, data_start_date, data_end_date, start_runtime, end_runtime, table_name) '
                   'VALUES (?,?,?,?,?,?,?)',
                  start_proc_time.strftime('%Y-%m-%d'),
                  cfg_load_method,
                  cfg_start_date.strftime('%Y-%m-%d'),
                  cfg_end_date.strftime('%Y-%m-%d'),
                  start_proc_time,
                  end_proc_time, 
                  table_name) 

            conn.commit()
        except:
            conn.rollback()
            print(sys.exc_info())
        finally:  
            cursor.close()

In [6]:
campaingn_opens_dataset = pd.DataFrame()
all_records = []
n_past_days = 120
params = {"page": page, "pagesize": pagesize, "orderfield": "email", "orderdirection": "asc" }
#desired_columns = ["campaignId", "EmailAddress", "Date", "CountryCode", "ListID"]
desired_columns = ["campaignId", "EmailAddress", "Date"]

for campaign_id in campaigns_on_active_period["CampaignID"]:

    url_campaign_opens = f"https://api.createsend.com/api/v3.2/campaigns/{campaign_id}/opens.json"
    df_campaign_email_opens = pd.DataFrame()
    for x in range(n_past_days):
        
         N_days_ago_datetime = get_datetime_Ndays_ago(x)
         # Pass N days if not full load
         params["date"] = N_days_ago_datetime
         response = requests.get(url_campaign_opens, headers={'Authorization': auth_header }, params = params) 
         #print(response.json())
         response_campaign_opens = response.json()
         if(len(response_campaign_opens["Results"]) == 0): continue
         #print("Data exists")
         response_dataframe = pd.DataFrame.from_dict(response_campaign_opens["Results"])
         response_dataframe["campaignId"] = campaign_id
         
         df_campaign_email_opens = df_campaign_email_opens.append(response_dataframe, ignore_index=True, sort=False) 
         
    
    all_records.append(df_campaign_email_opens)
    

    #print("All record length", len(all_records))
    if(len(all_records) > 0):
        campaingn_opens_dataset = pd.concat(all_records, axis=0, ignore_index=True, sort=False) 
        campaingn_opens_dataset = campaingn_opens_dataset[desired_columns]
        campaingn_opens_dataset = campaingn_opens_dataset.drop_duplicates()


print("Total Number of Records", campaingn_opens_dataset.shape)

Total Number of Records (10, 3)


In [39]:
##### Isolated Code for Degugging Purpose ######
campaign_email_action_list = ["opens", "clicks", "unsubscribes", "bounces", "spam"]

final_campaign_action_data = {"opens": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_opens"},
                              "clicks":{"data": pd.DataFrame(), "destination_table_name": "campaign_email_clicks"},
                              "unsubscribes": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_unsubscribes"},
                              "bounces": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_bounces"},
                              "spam": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_spam"} }

for action in campaign_email_action_list:
        table_name = final_campaign_action_data[action]["destination_table_name"]
        campaign_data = final_campaign_action_data[action]["data"]
        print(table_name)
        print(campaign_data)

campaign_email_opens
Empty DataFrame
Columns: []
Index: []
campaign_email_clicks
Empty DataFrame
Columns: []
Index: []
campaign_email_unsubscribes
Empty DataFrame
Columns: []
Index: []
campaign_email_bounces
Empty DataFrame
Columns: []
Index: []
campaign_email_spam
Empty DataFrame
Columns: []
Index: []
